In [1]:
import pandas as pd
import os

In [23]:
# read msa2county cross walk 
msa_county_cw = pd.read_excel('cbsa2fipsxw.xlsx')

# select only msa 
msa_county_cw = msa_county_cw[msa_county_cw['metropolitanmicropolitanstatis']=='Metropolitan Statistical Area']

# create county column
msa_county_cw.loc[:, 'countyfips'] = msa_county_cw['fipsstatecode'].apply(lambda x: str(x).zfill(2)) + msa_county_cw['fipscountycode'].apply(lambda x: str(x).zfill(3))

# select needed column
msa_county_cw = msa_county_cw[['cbsatitle', 'countyfips']]

# transform to dict
msa_county_cw = dict(zip(msa_county_cw['countyfips'], msa_county_cw['cbsatitle']))

In [2]:
# list all files need to be filtered
files = os.listdir('2024_Jan-May_monthly')

In [ ]:
for file in files:
    # read month mobility pattern
    df_mp = pd.read_csv('2024_Jan-May_monthly/'+files[0], dtype={'category':'category'}) # save memory use by pre-assign column type

    # get county fipscode for both poi and bg
    df_mp.loc[:, 'poi_county'] = df_mp['poi_cbg'].apply(lambda x: str(int(x)).zfill(12)[0:5])
    df_mp.loc[:, 'bg_county'] = df_mp['visitor_home_cbgs'].apply(lambda x: str(x).zfill(12)[0:5])

    # get msa for both poi and bg
    df_mp.loc[:, 'poi_msa'] = df_mp['poi_county'].map(msa_county_cw)
    df_mp.loc[:, 'bg_msa'] = df_mp['bg_county'].map(msa_county_cw)

    # filter data within each msa
    df_mp = df_mp[df_mp['poi_msa'] == df_mp['bg_msa']]

    # save file to disk
    df_mp.rename(columns={'bg_msa':'msa'}, inplace=True)
    df_mp = df_mp[['poi_cbg', 'category', 'visitor_home_cbgs', 'visitor_count','msa']]
    df_mp.to_csv('2024_monthly_msa/'+file)